# Theory:



## Motivation

A key challenges in accumulation of property insurance is to identify and draw boundary around areas of exposure, such as create and understand the concentrations of risk. This is most commonly achieved through accumulation management, or the monitoring of insured values (i.e. limits excluding all deductibles) in any one area to avoid an over-concentration of risk. Accumulation management evaluates impact on property by assessing concentrations in the worst case scenarios by assuming 100% damage. As all limits and deductibles are applied on total sum insured of the concentrated risks, it is crucial to know all those risks that are co-related to each other.

In traditional risk management approach the area of the concentration could be city, state, cresta, country or any user defined geographic grid. However in case terrorism risk, it is not safe to assume these political-geographic boundary as concentration of risk. It's mainly because of the following two reason:

* The scope of risk goes beyond any political-geographic boundary.
 
* The area of accumulation is very limited and it is less than the area of a city or even a pin code.
    Usually, accumulation of terrorism risk measure in terms of radius.
    For example, a potential bomb can affect maximum 250 meter radius (e.g. how much exposure is within 250 meters of point x?).
    Hence, we need a different approach to accumulate terrorism risk.

## Vendor Model (RMS) Approach

**Spider analysis**

In general, the goal of this analysis is to identify the “worst” areas in terms of accumulation, as defined by the analysis type. A Spider analysis identifies the area by itself. The area type can be a geographic area such as CRESTA zone, county, or postal code. The area type could also be a simple damage circle which assumes 100% ground-up.

The user selects a method of attack (maximum area of affect, i.e. in terms of radius of a circle). The user also specifies a threshold when running a spider analysis. This threshold limits the number of accumulation areas identified. Assume you want to know where the top 50 areas of greatest exposure concentration lie assuming an accumulation within a 250 meter radius circle. This can be accomplished by performing a Spider analysis. The user could also specify monetary thresholds such as $500 million in an area.

RiskLink uses a fixed grid system that blankets the earth. For a 100% simple damage circle, the default grid size is 25% of the radius selected. For example, if a simple damage circle is defined by a radius of 400 meters, the grid size would be 100 meters by 100 meters (400 * 25%).

For each cell in the grid system, a circle is drawn that represents the accumulation area. The centroid of each circle is placed at the center of each cell. As each circle is drawn the exposure for each circle is accumulated. This process continues until a circle is drawn for each cell. Once all circles have been drawn, the circle with the highest accumulation is identified, any areas that overlap with the highest accumulated area are removed. This is called event generation and in this process RiskLink throws away few exposure.
The final step in the spider analysis, after eliminating overlapping circles, is to give the accumulations a descriptive name. All events are named based on the location closest to the center of the circle. Let's say the closest location from center of an accumulated circle is located in the country "US", state "California", city "Foothill Ranch" and postal code "91916". Therefore, the name of the event would be “Foothill Ranch, CA 92610, US”.

## Our Approach (TRAM)

This method is very much similar to RiskLink except the methodology for fining accumulated circles. While RMS uses Spider algorithm to do so, we are using a supervise algorithm called Agglomerative Hierarchical clustering. Hierarchical clustering involves creating clusters that have a predetermined ordering from top to bottom.

For example, all files and folders on the hard disk are organized in a hierarchy. In agglomerative or bottom-up clustering method first each location is assigned to its own cluster.
Then, compute the similarity (e.g., distance) between each of the clusters and then join the two most similar clusters; any two clusters with minimum distance to each other. Finally, repeat steps 2 and 3 until there is only a single cluster left. In theory, it can also be done by initially grouping all the observations into one cluster, and then successively splitting these clusters.

Before any clustering is performed, a proximity matrix is determined which contains the distance between each point using a distance function. Then, the matrix is updated to display the distance between each cluster. There are three methods differ in how the distance between each cluster is measured. The one we have used hear is called "complete linkage".
In complete linkage hierarchical clustering, the distance between two clusters is defined as the longest distance between two points in each cluster.

Finally, an agglomeration tree will be formed in the end of this algorithm (as shown below). In this tree all the cluster are shown at x axis, and number of iteration y axis.
Additionally we know the distance between any two clusters at Nth iteration.

As we have used complete leakage method, if we cut this tree from ground till X height then we will get clusters with maximum 2X distance between any two locations.
This approach also ensure every location will be a part of a single cluster.

###### load sample data

In [3]:
# File location and type
file_location = "/FileStore/tables/data.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [4]:
from pyspark.sql.functions import col, lit, concat_ws, udf, struct, first, pandas_udf, PandasUDFType
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, FloatType, StructType
from pyspark.sql import SQLContext
from pyspark import SparkContext
from math import ceil
from scipy.cluster.hierarchy import fcluster, linkage
from scipy.spatial.distance import pdist
from geopy import distance

###### Input from user blast zone radius, peril & portfolio

In [6]:
blast_radius = sc.broadcast(200)
peril = sc.broadcast("202")
portfolio = sc.broadcast(["3"])
max_items_for_clustering = sc.broadcast(50000)

###### If geo encoding is more than zip level, it dosen't make seance to include in the clusturing process. As RMS, we are also ignoring address match greater than 5 (i.e. considering only upto postal code granularity). We are not considering "number of building" as a measure of aggregated location.

In [8]:
encoding_type_list = sc.broadcast(["EXACT", "EXACT_STREET_ADDRESS", "ZIP"])
# encoding_type_list = ["ADMIN0", "ADMIN1", "ADMIN2", "CRESTA", "EXACT", "EXACT_STREET_ADDRESS", "PLACE_CITY", "SUB_CRESTA", "ZIP"]

###### Filtering dataframe for the user input peril & portfolio

In [10]:
df = df\
  .filter((col("pf_version_id").isin(portfolio.value)) & (col("peril_type_cd") == peril.value) & (col("encoding_quality_desc").isin(encoding_type_list.value))) \
  .select("pf_version_id", "admin_0_name", "ins_itm_id", "latitude", "longitude") \
  .dropDuplicates() \
  .withColumn("identifier", concat_ws("_", col("pf_version_id"), col("admin_0_name"))) \
  .withColumn("cluster", lit(None).cast(FloatType())) \
  .repartitionByRange(100, "identifier")

df.persist()
display(df)

pf_version_id,admin_0_name,ins_itm_id,latitude,longitude,identifier,cluster
3,AR,912784,-33.1478,-68.48569,3_AR,null
3,AR,915244,-32.897442,-68.829727,3_AR,null
3,AR,913918,-26.829479,-65.214272,3_AR,null
3,AE,962487,24.964292,55.057713,3_AE,null
3,AF,861730,34.53091,69.136757,3_AF,null
3,AR,915192,-34.623661,-60.07975,3_AR,null
3,AR,912450,-32.897442,-68.829727,3_AR,null
3,AR,915378,-29.15,-59.85,3_AR,null
3,AT,963737,47.362687,11.720428,3_AT,null
3,AE,859164,24.476431,54.37051,3_AE,null


###### pandas grouped map udf

In [12]:
@pandas_udf("pf_version_id int, admin_0_name string, ins_itm_id int, latitude double, longitude double, identifier string, cluster float", PandasUDFType.GROUPED_MAP) 
def create_cluster(data):
  if data.shape[0] == 1:
    clust = data.cluster
  else:
    # Creating a distance matrix considering geo distance two LatLong
    distanceMatrix = pdist(data[["latitude","longitude"]], lambda u, v: distance.geodesic(u, v).meters)
    # Linkage methods are used to compute the distance d(s,t) between two clusters
    linkMatrix = linkage(distanceMatrix, method='complete')
    # Cutting the dendogram tree from leaf (i.e. distance 0) to blast diameter
    clust = fcluster(linkMatrix, blast_radius.value * 2, "distance")
  return data.assign(cluster = clust)

In [13]:
df_cluster = df.groupBy("identifier").apply(create_cluster)

In [14]:
df_cluster.write.format("csv").save("/FileStore/tables/data_cluster.csv")

java.net.ConnectException: Connection refused
	at sun.nio.ch.SocketChannelImpl.checkConnect(Native Method)
	at sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:717)
	at sun.nio.ch.SocketAdaptor.connect(SocketAdaptor.java:111)
	at org.eclipse.jetty8.client.SelectConnector.startConnection(SelectConnector.java:82)
	at org.eclipse.jetty8.client.HttpDestination.startNewConnection(HttpDestination.java:292)
	at org.eclipse.jetty8.client.HttpDestination.doSend(HttpDestination.java:597)
	at org.eclipse.jetty8.client.HttpDestination.send(HttpDestination.java:532)
	at org.eclipse.jetty8.client.HttpClient.send(HttpClient.java:180)
	at com.databricks.rpc.JettyClient.$anonfun$sendRawExchange$2(JettyClient.scala:286)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
	at com.databricks.rpc.AbstractJettyClient.withAttributionContext(BaseJettyClient.scala:81)
	at com.databricks.rpc.JettyClient.sendRawExchange(JettyClient.scala:270)
	at com.databricks.rpc.JettyClient.sendExchange(JettyClient.scala:186)
	at com.databricks.rpc.JettyClient.sendAsync(JettyClient.scala:142)
	at com.databricks.rpc.DynamicJettyClient.sendAsync(BaseJettyClient.scala:565)
	at com.databricks.rpc.BoundRPCClient.sendAsync(BoundRPCClient.scala:54)
	at com.databricks.backend.daemon.driver.DriverClient.send(DriverClient.scala:98)
	at com.databricks.backend.daemon.driver.DriverClient.$anonfun$poll$1(DriverClient.scala:399)
	at com.databricks.util.FutureUtils$.retry(FutureUtils.scala:87)
	at com.databricks.util.FutureUtils$$anonfun$retry$2.applyOrElse(FutureUtils.scala:91)
	at com.databricks.util.FutureUtils$$anonfun$retry$2.applyOrElse(FutureUtils.scala:89)
	at scala.concurrent.Future.$anonfun$recoverWith$1(Future.scala:417)
	at scala.concurrent.impl.Promise.$anonfun$transformWith$1(Promise.scala:41)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at com.databricks.threading.NamedExecutor$$anon$1.$anonfun$run$1(NamedExecutor.scala:317)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
	at com.databricks.threading.NamedExecutor.withAttributionContext(NamedExecutor.scala:256)
	at com.databricks.threading.NamedExecutor$$anon$1.run(NamedExecutor.scala:317)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)